In [1]:
import nilearn as nl
import glob
import numpy as np
import pandas as pd

In [2]:
data = glob.glob('/home/davide/Desktop/DAVE/BHG_Padova/data/*/anat/*T1*')

In [4]:
t1 = nl.image.load_img(data[0]) 
t1 = t1.get_data()
all_t1 = np.zeros([len(data), t1.shape[0],  t1.shape[1],  t1.shape[2]])
for subj in range(len(data)):
    t12use = nl.image.load_img(data[subj]) 
    t12use = t12use.get_data()
    all_t1[subj,:,:,:] = t12use


/home/davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
/home/davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


In [5]:
all_t1.shape

(226, 240, 240, 220)

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [15]:
demographic = pd.read_csv('participants.tsv', sep='\t')
demographic['handedness'] = demographic['handedness'].fillna(0)
demographic['handedness'].replace(['right','left', 'ambidextrous'],[0,1,1],inplace=True)
demographic.head()

,participant_id,age,sex,BMI,handedness,education_category,raven_score,NEO_N,NEO_E,NEO_O,NEO_A,NEO_C
0,sub-0001,25.50,M,21.0,0,academic,33.0,23,40,52,47,32
1,sub-0002,23.25,F,22.0,0,academic,19.0,22,47,34,53,46
2,sub-0003,25.00,F,23.0,0,applied,29.0,26,42,37,48,48
3,sub-0004,20.00,F,18.0,0,academic,24.0,32,42,36,48,52
4,sub-0005,24.75,M,27.0,0,academic,24.0,32,51,41,51,53


In [16]:
X = np.reshape(all_t1, (all_t1.shape[0], all_t1.shape[1], all_t1.shape[2]*all_t1.shape[3]))
X.shape


(226, 240, 52800)

In [ ]:
y = np.array(demographic['handedness'])

y = to_categorical(y)


X = X.reshape(-1, 240, 52800, 1)
                

input_shape = (240, 52800)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(2))
model.add(Activation('sigmoid'))
#model.compile(loss='binary_crossentropy',
 #             optimizer='adam',
  #            metrics=['accuracy'])

opt = SGD(lr=0.001)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=20, validation_split=0.3)

Train on 158 samples, validate on 68 samples
Instructions for updating:
Use tf.cast instead.
